In [59]:
import torch
import torch.optim as optim
import numpy as np

torch.manual_seed(0)

In [60]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [61]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c : i for i, c in enumerate(char_set)}

In [62]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10 # 어떤 값이든 가능
learning_rate = 0.1

In [63]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i+1 : i + sequence_length + 1]
    
    print('%03d'%i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])
    
x_one_hot = [np.eye(dic_size)[x] for x in x_data]

000 if you wan -> f you want
001 f you want ->  you want 
002  you want  -> you want t
003 you want t -> ou want to
004 ou want to -> u want to 
005 u want to  ->  want to b
006  want to b -> want to bu
007 want to bu -> ant to bui
008 ant to bui -> nt to buil
009 nt to buil -> t to build
010 t to build ->  to build 
011  to build  -> to build a
012 to build a -> o build a 
013 o build a  ->  build a s
014  build a s -> build a sh
015 build a sh -> uild a shi
016 uild a shi -> ild a ship
017 ild a ship -> ld a ship,
018 ld a ship, -> d a ship, 
019 d a ship,  ->  a ship, d
020  a ship, d -> a ship, do
021 a ship, do ->  ship, don
022  ship, don -> ship, don'
023 ship, don' -> hip, don't
024 hip, don't -> ip, don't 
025 ip, don't  -> p, don't d
026 p, don't d -> , don't dr
027 , don't dr ->  don't dru
028  don't dru -> don't drum
029 don't drum -> on't drum 
030 on't drum  -> n't drum u
031 n't drum u -> 't drum up
032 't drum up -> t drum up 
033 t drum up  ->  drum up p
034  drum up p

In [64]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [65]:
# declare RNN + FC
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim,
                                num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(dic_size, hidden_size, 2)

In [66]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [67]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
            
    print(predict_str)
    print()

dhsw'd''dd'ddhdwhh'''ds'dhd'dhhd'ddddd''dddddhdh'ddhh'dh'ddhhddhdd'shdhdh''''''''d'dddh'dd'd'h'h'hhhd''d'''''dhh'whd'ddhh'ddhddsh'dh'hdhhdd'dh'h'dd.h'd'dh'''hdhhh'd'hdd'hh'dh'dh'd

                                                                                                                                                                                   

 eto oea oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

  ubykukgkspmssmmshsppsmshsmspshshsmspsmshsmmshsmpsmhscsmspsrckksmsmssmchsmshsppshshspspshspsmmscssbmhsps spshshshshsmshsmspsmckbspsmspsmsm,shsmsstmshsmshsmspsmu scspsmsssspesshcp

                                                                                                                                                                                   

  t    u                    d       d                                                          

g wou want to luild a ship, don't drum up people together to collect dood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sear

g wou want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the eean

g wou want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sean

g wou want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sean

g wou want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sea.

g wou want to build a ship, don't drum up people together to collect wood and don't assign them